# Introduction
This colab worksheet provides a starting point for Task 1 (the natural language processing assignment).

# Data Loading

In [1]:
# Download the data stored in a csv file from one of these two locations
# If you're running all your experiments
# on a machine at home rather than using colab, then make sure you save it
# rather than repeatedly downloading it.

!wget "https://sussex.box.com/shared/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n" -O spam_detection_training_data.csv

# The test images (without points)
!wget "https://sussex.box.com/shared/static/2a1am3esr4yzjmqr172vua1t41a846e0" -O spam_detection_test_data.csv

--2025-05-15 09:22:34--  https://sussex.box.com/shared/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n
Resolving sussex.box.com (sussex.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to sussex.box.com (sussex.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n [following]
--2025-05-15 09:22:34--  https://sussex.box.com/public/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n
Reusing existing connection to sussex.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://sussex.app.box.com/public/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n [following]
--2025-05-15 09:22:34--  https://sussex.app.box.com/public/static/qdmzn1esyjk6hwa4gy9y5y0tp0fpcr0n
Resolving sussex.app.box.com (sussex.app.box.com)... 74.112.186.157, 2620:117:bff0:12d::
Connecting to sussex.app.box.com (sussex.app.box.com)|74.112.186.157|:443... connected.
HTTP request sent, awaiting resp

# Check the data downloaded correctly
If any of these assertions fail, redownload the data

In [2]:
def confirm_checksum(filename, true_checksum):
  import subprocess
  checksum = subprocess.check_output(['shasum',filename]).decode('ascii')
  assert checksum.split(' ')[0] == true_checksum, 'Checksum does not match for ' + filename + ' redownload the data.'

confirm_checksum('spam_detection_training_data.csv', '807818a04b1f14412767e0929014fe0279047188')
confirm_checksum('spam_detection_test_data.csv', '93556bbad693968096613011355fa490d1fad4d5')



# Load the data

In [3]:
import pandas as pd

# Load the data using np.load
data = pd.read_csv('spam_detection_training_data.csv')

# Extract the text
text = data['text'].values
# and the labels
labels = data['label'].values

print(text.shape, labels.shape)

test_data = pd.read_csv('spam_detection_test_data.csv')
test_text = test_data['text'].values
print(test_text.shape)

(3619,) (3619,)
(1552,)


# Data Visualisation
Here's an example of how to display the text based on its label.

In [4]:
def print_text(text, label):
  if label == 0:
    print (text, '\nis not spam!')
  else:
    print (text, '\nis spam!')

import numpy as np
idx = np.random.randint(0, text.shape[0])
print_text(text[idx], labels[idx])

Subject: skip the doctor - buy meds online
up to 80 % savings on xanax , valium , cialis , viagra , tylenol 3 & more
here
for
email removal , go here .
begonia cookie adopt bilinear genevieve reck conducive patron playoff complaisant cashew eat cater appeasable thrips frostbite distributor walleralliance dug myel bureaucracy offbeat patrolman billfold bifocal concord eisner burgundian davidson baronet proust emperor anderson baldpate captivateurinary debenture shareown catkin philanthrope credit stabile basel clotho soluble epitaxy hint hirsute demountable broody chaplaincy rime vanadium dempsey armageddon farfetched klan populace led cellar durer diversionary windbreak astor propel bonneville antenna honest portmanteaudiopter workload yvette crusty mathematician partridge covet cartwheel kennan michael cornbread imitate corbel pope zion 
is spam!


# Calculating Confusion Matrix and exporting results

In [20]:
from sklearn.metrics import confusion_matrix

def my_confusion_matrix(true_label, pred_label):
  """
  Calculate the confusion matrix for your predicted labels. See https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
  :param pred_label: Array of predicted labels
  :param true_label: Array of corresponding ground truth (test) labels
  :return: Confusion matrix whose i-th row and j-th column entry indicates the number of samples with true label being i-th class and predicted label being j-th class.
  """
  return confusion_matrix(true_label, pred_label)


In [21]:
def save_as_csv(pred_labels, location = '.'):
    """
    Save the labels out as a .csv file
    :pred_labels: numpy array of shape (no_test_labels,) to be saved
    :param location: Directory to save results.csv in. Default to current working directory
    """
    assert pred_labels.shape[0]==1552, 'wrong number of labels, should be 1552 test labels'
    np.savetxt(location + '/results_task1.csv', pred_labels, delimiter=',')


In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(y_true, y_pred):
    """
    Calculate and print evaluation metrics.
    :param y_true: Ground truth labels
    :param y_pred: Predicted labels
    """
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print("Accuracy:", round(acc, 3))
    print("Precision:", round(prec, 3))
    print("Recall:", round(rec, 3))
    print("F1 Score:", round(f1, 3))


#Convert the training dataa  into tfidf features
tfidf = TfidfVectorizer()   #Create vectoriser
X_train = tfidf.fit_transform(text)  #train vocabulary  and transform training data

#train the logistic regression model using the  tfidf features
model = LogisticRegression()
#model = match training data and labels so that  it knows what type of data gives what label
model.fit(X_train, labels)

# Evaluate the model on training data
train_preds = model.predict(X_train)
evaluate_model(labels, train_preds)


#Test Data
#transform function on,test  data using the same vectorizer
X_test = tfidf.transform(test_text)

#predict the labels for the test data - spam/no spam
pred_labels = model.predict(X_test)

#save the predictions
save_as_csv(pred_labels)


import numpy as np

# Make predictions on the test data
pred_labels = model.predict(X_test)

# Show predictions for 10 random test messages
for _ in range(15):
    i = np.random.randint(0, len(test_text))  # pick a random index
    print("Test Message:", test_text[i])
    print("Predicted Label:", "Spam" if pred_labels[i] == 1 else "Not Spam")


    print("----------------------------------------------------------")



Accuracy: 0.994
Precision: 0.985
Recall: 0.996
F1 Score: 0.991
Test Message: Subject: 
the permanent fix to penis growth
limited offer :
add atleast 4 inches or get your money back !
- - - - > visit us to learn more
no more offers

Predicted Label: Spam
----------------------------------------------------------
Test Message: Subject: lacy
hey - - just thought i ' d let you know that lacy went to school this morning - pale & not feeling well - - stomach still hurts , but she went ahead . she ' s stressing me out big time . she acted like she couldn ' t hardly move , but i came back in the house & she was talking fine on the telephone . but i ask her questions about how she feels & she is just about dead . i don ' t know about her . but she looked pale . blood sugar was a little low this morning . i woke her up at 2 & had her check it & it was good . so i got a little more rest . i don ' t think she even woke up enough to remember it . i don ' t think the little job is good for her if sh